# RTI Race code

* Exploring options to determine processing rules for RTI Race code

In [1]:
## Load packages ----
import numpy as np
import pandas as pd
import sshtunnel
import psycopg2 as pg
import os
import seaborn as sns
import matplotlib.pyplot as plt

/n/home_fasse/nsardana/.conda/envs/dorieh/lib/python3.10/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
## Open ssh tunnel to DB host ----
tunnel = sshtunnel.SSHTunnelForwarder(
    ('nsaph.rc.fas.harvard.edu', 22),
    ssh_username=f'{os.environ["MY_NSAPH_SSH_USERNAME"]}',
    ssh_private_key=f'{os.environ["HOME"]}/.ssh/id_rsa', 
    ssh_password=f'{os.environ["MY_NSAPH_SSH_PASSWORD"]}', 
    remote_bind_address=("localhost", 5432)
)

tunnel.start()

In [3]:
## Open connection to DB ----
connection = pg.connect(
    host='localhost',
    database='nsaph2',
    user=f'{os.environ["MY_NSAPH_DB_USERNAME"]}',
    password=f'{os.environ["MY_NSAPH_DB_PASSWORD"]}', 
    port=tunnel.local_bind_port
)

## Data munging

In [4]:
sql_query = f"""SELECT * FROM medicare.beneficiaries LIMIT 10;"""

In [5]:
sql_query

'SELECT * FROM medicare.beneficiaries LIMIT 10;'

In [6]:
## Request query ----
df = pd.read_sql_query(sql_query, connection, index_col = 'bene_id').reset_index()
df

,bene_id,dob,yob,dod,first_enrollment_year,last_enrollment_year,all_enrollment_years,race,race_rti,sex,orec,discrepancies,dob_latest,yob_latest,dod_earliest,bene
0,lllllll88XlU0Xo,1944-12-13,1944,None,2009,2018,"[2009, 2010, 2011, 2012, 2013, 2014, 2015, 201...",1,5,1,0,1,None,1945,None,8892460531093126377
1,lllllll88XlU0XO,1945-03-23,1945,None,2010,2018,"[2010, 2011, 2012, 2013, 2014, 2015, 2016, 201...",2,5,2,0,1,None,1946,None,-5796525173252050234
2,lllllll88XlU0XS,1944-09-24,1944,None,2009,2018,"[2009, 2010, 2011, 2012, 2013, 2014, 2015, 201...",1,5,1,0,1,None,1945,None,-9208669245228246855
3,lllllll88XlU0XU,1961-11-05,1961,None,2009,2018,"[2009, 2010, 2011, 2012, 2013, 2014, 2016, 201...",1,5,2,1,0,None,1962,None,-7133705442536682347
4,lllllll88XlU0XX,1944-12-02,1944,None,2009,2018,"[2009, 2010, 2011, 2012, 2013, 2014, 2015, 201...",1,5,1,0,1,None,1945,None,6912619001498574969
5,lllllll88XlU400,1944-10-11,1944,None,2009,2018,"[2009, 2010, 2011, 2012, 2013, 2014, 2015, 201...",1,"1,3",2,0,1,None,1945,None,-445196388116607587
6,lllllll88XlU404,1973-11-11,1973,None,2009,2018,"[2009, 2010, 2011, 2012, 2013, 2014, 2016, 201...",1,1,1,1,0,None,1974,None,-5778668796759057592
7,lllllll88XlU407,1944-12-08,1944,None,2009,2018,"[2009, 2010, 2011, 2012, 2013, 2014, 2015, 201...",1,1,1,0,1,None,1945,None,-4149042624558812527
8,lllllll88XlU408,1945-01-02,1945,None,2010,2018,"[2010, 2011, 2012, 2013, 2014, 2015, 2016, 201...",1,1,2,0,1,None,1946,None,-8726804549402563331
9,lllllll88XlU40l,1945-02-17,1945,None,2010,2018,"[2010, 2011, 2012, 2013, 2014, 2015, 2016, 201...",1,1,2,0,1,None,1946,None,5826909233909847751
